# Generate interactive html visualizations of Ball Mapper graphs.
Allows for switching between multiple coloring functions using a dropdown menu

In [1]:
import networkx as nx

import numpy as np
import pandas as pd

from bokeh.io import show, save
from bokeh.models import (
    Plot,
    Range1d,
    MultiLine,
    Circle,
    TapTool,
    OpenURL,
    HoverTool,
    CustomJS,
    Slider,
    Column,
    StaticLayoutProvider,
    TapTool,
    WheelZoomTool,
    PanTool,
    ResetTool,
    SaveTool,
    FixedTicker,
    LinearColorMapper,
    LogColorMapper,
    ColorBar,
    BasicTicker,
    LogTicker,
    Dropdown,
    RadioButtonGroup,
)
from bokeh.plotting import figure, from_networkx
from bokeh.io import export_svgs

import matplotlib.pyplot as plt

from matplotlib import colormaps as cm
from matplotlib.colors import to_hex
from pyballmapper import BallMapper
from pyballmapper.plotting import graph_GUI

In [2]:
coloring_df = pd.read_csv("data/June24knotinfo.csv", index_col=0).reset_index(drop=True)

coloring_df.columns = coloring_df.columns.str.replace(" ", "_")
coloring_df.columns = coloring_df.columns.str.replace("-", "_")

coloring_df.replace(to_replace="Yes", value=1, inplace=True)
coloring_df.replace(to_replace="No", value=0, inplace=True)
coloring_df.replace(to_replace="Y", value=1, inplace=True)
coloring_df.replace(to_replace="N", value=0, inplace=True)
coloring_df.dropna(axis=1, inplace=True)

coloring_df["Signature_mod4"] = coloring_df.Signature % 4

coloring_df

/var/folders/4s/_33kvc6507v69vbr_vy9xsjc0000gq/T/ipykernel_17208/4062354001.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  coloring_df.replace(to_replace="No", value=0, inplace=True)
/var/folders/4s/_33kvc6507v69vbr_vy9xsjc0000gq/T/ipykernel_17208/4062354001.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  coloring_df.replace(to_replace="Y", value=1, inplace=True)
/var/folders/4s/_33kvc6507v69vbr_vy9xsjc0000gq/T/ipykernel_17208/4062354001.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be remo

,Fibered,Crossing_Number,Unknotting_Number,Genus_3D,Crosscap_Number,Bridge_Index,Braid_Index,Signature,Thurston_Bennequin_Number,Arc_Index,...,Genus_4D_(Top.),Determinant,Rasmussen_s,Ozsvath_Szabo_tau,Arf_Invariant,Turaev_Genus,L_space,Epsilon,Ropelength,Signature_mod4
0,1,3,1,1,1,2,2,-2,[1][-6],5,...,1,3,2,1,1,0,1,1,32.74360,2
1,1,4,1,1,2,2,3,0,[-3][-3],6,...,1,5,0,0,1,0,0,0,42.08870,0
2,1,5,2,2,1,2,2,-4,[3][-10],7,...,2,5,4,2,1,0,1,1,47.20160,0
3,0,5,1,1,2,2,3,-2,[1][-8],7,...,1,7,2,1,0,0,0,1,49.47010,2
4,0,6,1,1,2,2,4,0,[-3][-5],8,...,0,9,0,0,0,0,0,0,56.70580,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2972,0,12,1,2,4,3,5,0,[-4][-8],12,...,1,81,0,0,0,1,0,0,96.96986,0
2973,0,12,[1;2],3,[2;5],3,5,0,[-5][-7],12,...,1,149,0,0,1,1,0,0,100.51760,0
2974,0,12,2,3,[2;5],3,5,-2,[-2][-10],12,...,1,159,2,1,0,1,0,1,100.89584,2
2975,1,12,2,4,[2;5],3,3,-4,[2][-14],12,...,2,125,4,2,1,1,0,1,95.93502,0


In [3]:
to_keep = [
    c for c in coloring_df.columns if coloring_df.dtypes[c] != np.dtype("object")
]
to_keep

['Fibered',
 'Crossing_Number',
 'Genus_3D',
 'Bridge_Index',
 'Braid_Index',
 'Signature',
 'Arc_Index',
 'Genus_4D',
 'Determinant',
 'Rasmussen_s',
 'Ozsvath_Szabo_tau',
 'Arf_Invariant',
 'L_space',
 'Epsilon',
 'Ropelength',
 'Signature_mod4']

In [4]:
coloring_df = coloring_df[to_keep]
coloring_df

,Fibered,Crossing_Number,Genus_3D,Bridge_Index,Braid_Index,Signature,Arc_Index,Genus_4D,Determinant,Rasmussen_s,Ozsvath_Szabo_tau,Arf_Invariant,L_space,Epsilon,Ropelength,Signature_mod4
0,1,3,1,2,2,-2,5,1,3,2,1,1,1,1,32.74360,2
1,1,4,1,2,3,0,6,1,5,0,0,1,0,0,42.08870,0
2,1,5,2,2,2,-4,7,2,5,4,2,1,1,1,47.20160,0
3,0,5,1,2,3,-2,7,1,7,2,1,0,0,1,49.47010,2
4,0,6,1,2,4,0,8,0,9,0,0,0,0,0,56.70580,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2972,0,12,2,3,5,0,12,1,81,0,0,0,0,0,96.96986,0
2973,0,12,3,3,5,0,12,1,149,0,0,1,0,0,100.51760,0
2974,0,12,3,3,5,-2,12,1,159,2,1,0,0,1,100.89584,2
2975,1,12,4,3,3,-4,12,2,125,4,2,1,0,1,95.93502,0


In [12]:
TITLE = "Jones up to 12 crossing"

jones_df = pd.read_csv(
    "data/jones_knotinfo.csv",
    index_col=0,
)
jones_df

,t-13,t-12,t-11,t-10,t-9,t-8,t-7,t-6,t-5,t-4,...,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16
knot_id,,,,,,,,,,,,,,,,,,,,,
3_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5_1,0,0,0,0,0,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
5_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12n_884,0,0,0,0,0,0,0,0,0,0,...,-1,0,0,0,0,0,0,0,0,0
12n_885,0,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
12n_886,0,0,0,0,0,0,0,0,0,0,...,8,-3,0,0,0,0,0,0,0,0


## BallMapper

In [24]:
EPS = 10
jones_bm = BallMapper(X=jones_df.to_numpy(), eps=EPS, coloring_df=coloring_df)

jones_gui = graph_GUI(
    jones_bm.Graph,
    cm.get_cmap("jet"),
    ["Signature"],
    render_iterations=1000,
)
jones_gui.color_by_variable("Signature")

show(jones_gui.plot)

color by variable Signature 
MIN_VALUE: -8.000, MAX_VALUE: 6.000


## Interactive visualization

In [25]:
## compute all colors
coloring_variables_dict = dict()
for var in coloring_df.columns:
    coloring_variables_dict[var] = dict()

In [26]:
coloring_variables_dict

{'Fibered': {},
 'Crossing_Number': {},
 'Genus_3D': {},
 'Bridge_Index': {},
 'Braid_Index': {},
 'Signature': {},
 'Arc_Index': {},
 'Genus_4D': {},
 'Determinant': {},
 'Rasmussen_s': {},
 'Ozsvath_Szabo_tau': {},
 'Arf_Invariant': {},
 'L_space': {},
 'Epsilon': {},
 'Ropelength': {},
 'Signature_mod4': {}}

In [27]:
# set each variable palette

for var in coloring_variables_dict:
    if len(coloring_df[var].unique()) > 2:
        coloring_variables_dict[var]["palette"] = cm.get_cmap("jet")
        coloring_variables_dict[var]["style"] = "continuous"
    else:
        coloring_variables_dict[var]["palette"] = cm.get_cmap("Reds")
        coloring_variables_dict[var]["style"] = "discrete"

In [28]:
G = jones_bm.Graph

for var in coloring_variables_dict:
    print(var)
    MIN_VALUE = 10000
    MAX_VALUE = -10000

    for node in G.nodes:
        if G.nodes[node][var] > MAX_VALUE:
            MAX_VALUE = G.nodes[node][var]
        if G.nodes[node][var] < MIN_VALUE:
            MIN_VALUE = G.nodes[node][var]

    coloring_variables_dict[var]["max"] = MAX_VALUE
    coloring_variables_dict[var]["min"] = MIN_VALUE

    for node in G.nodes:
        if not pd.isna(G.nodes[node][var]):
            color_id = (G.nodes[node][var] - MIN_VALUE) / (MAX_VALUE - MIN_VALUE)
            if coloring_variables_dict[var]["style"] == "log":
                color_id = (np.log10(G.nodes[node][var]) - np.log10(MIN_VALUE)) / (
                    np.log10(MAX_VALUE) - np.log10(MIN_VALUE)
                )
            G.nodes[node]["{}_color".format(var)] = to_hex(
                coloring_variables_dict[var]["palette"](color_id)
            )
        else:
            G.nodes[node]["{}_color"] = "black"

for node in G.nodes:
    G.nodes[node]["current_color"] = "white"

Fibered
Crossing_Number
Genus_3D
Bridge_Index
Braid_Index
Signature
Arc_Index
Genus_4D
Determinant
Rasmussen_s
Ozsvath_Szabo_tau
Arf_Invariant
L_space
Epsilon
Ropelength
Signature_mod4


In [29]:
coloring_variables_dict

{'Fibered': {'palette': <matplotlib.colors.LinearSegmentedColormap at 0x1697746e0>,
  'style': 'discrete',
  'max': 1.0,
  'min': 0.0},
 'Crossing_Number': {'palette': <matplotlib.colors.LinearSegmentedColormap at 0x169ab8e30>,
  'style': 'continuous',
  'max': 12.0,
  'min': 10.781609195402298},
 'Genus_3D': {'palette': <matplotlib.colors.LinearSegmentedColormap at 0x1697bac60>,
  'style': 'continuous',
  'max': 5.0,
  'min': 2.0},
 'Bridge_Index': {'palette': <matplotlib.colors.LinearSegmentedColormap at 0x169aa65a0>,
  'style': 'continuous',
  'max': 3.2,
  'min': 2.6923076923076925},
 'Braid_Index': {'palette': <matplotlib.colors.LinearSegmentedColormap at 0x16970ede0>,
  'style': 'continuous',
  'max': 6.0,
  'min': 3.0},
 'Signature': {'palette': <matplotlib.colors.LinearSegmentedColormap at 0x169aa7020>,
  'style': 'continuous',
  'max': 6.0,
  'min': -8.0},
 'Arc_Index': {'palette': <matplotlib.colors.LinearSegmentedColormap at 0x169ea7500>,
  'style': 'continuous',
  'max': 14

In [30]:
def create_colorbar(style, palette, low, high):

    if style == "continuous":
        # continuous colorbar
        num_ticks = 100
        color_mapper = LinearColorMapper(
            palette=[
                to_hex(palette(color_id)) for color_id in np.linspace(0, 1, num_ticks)
            ],
            low=low,
            high=high,
        )

        return ColorBar(
            color_mapper=color_mapper,
            major_label_text_font_size="14pt",
            label_standoff=12,
        )

    elif style == "log":
        # log colorbar
        num_ticks = 100
        color_mapper = LogColorMapper(
            palette=[
                to_hex(palette(color_id)) for color_id in np.linspace(0, 1, num_ticks)
            ],
            low=low,
            high=high,
        )

        log_ticks = LogTicker(mantissas=[1, 2, 3, 4, 5], desired_num_ticks=10)

        return ColorBar(
            color_mapper=color_mapper,
            major_label_text_font_size="14pt",
            label_standoff=12,
            ticker=log_ticks,
        )

    elif style == "discrete":
        # discrete colorbar

        if var in ["signature", "s_invariant"]:
            step = 2
            ticks = [i for i in range(int(low) // 2 * 2, int(high) // 2 * 2 + 1, step)]
        else:
            step = 1
            ticks = [i for i in range(int(low), int(high) + 1)]

        print(ticks)

        color_mapper = LinearColorMapper(
            palette=[
                to_hex(palette(color_id)) for color_id in np.linspace(0, 1, len(ticks))
            ],
            low=low - step / 2,
            high=high + step / 2,
        )

        color_ticks = FixedTicker(ticks=ticks)

        print(ticks)

        return ColorBar(
            color_mapper=color_mapper,
            major_label_text_font_size="14pt",
            label_standoff=12,
            ticker=color_ticks,
        )

In [31]:
G.nodes[1].keys()

dict_keys(['landmark', 'points covered', 'size', 'Fibered', 'Crossing_Number', 'Genus_3D', 'Bridge_Index', 'Braid_Index', 'Signature', 'Arc_Index', 'Genus_4D', 'Determinant', 'Rasmussen_s', 'Ozsvath_Szabo_tau', 'Arf_Invariant', 'L_space', 'Epsilon', 'Ropelength', 'Signature_mod4', 'size rescaled', 'color', 'Fibered_color', 'Crossing_Number_color', 'Genus_3D_color', 'Bridge_Index_color', 'Braid_Index_color', 'Signature_color', 'Arc_Index_color', 'Genus_4D_color', 'Determinant_color', 'Rasmussen_s_color', 'Ozsvath_Szabo_tau_color', 'Arf_Invariant_color', 'L_space_color', 'Epsilon_color', 'Ropelength_color', 'Signature_mod4_color', 'current_color'])

In [32]:
for node in G.nodes:
    del G.nodes[node]["points covered"]

In [33]:
plot = Plot(
    width=1000,
    height=750,
    x_range=Range1d(-2, 2),
    y_range=Range1d(-2, 2),
    # sizing_mode="stretch_both",
    toolbar_location="right",
    output_backend="svg",
    title=TITLE,
)

tooltips = [("index", "@index"), ("size", "@size")]

tooltips += [
    (name.replace("_", " "), "@{}".format(name)) for name in coloring_variables_dict
]

node_hover_tool = HoverTool(tooltips=tooltips)
zoom_tool = WheelZoomTool()
plot.add_tools(PanTool(), node_hover_tool, zoom_tool, ResetTool(), SaveTool())
plot.toolbar.active_scroll = zoom_tool

graph_renderer = from_networkx(
    G,
    nx.spring_layout,
    seed=42,
    scale=1,
    center=(0, 0),
    # k=10 / np.sqrt(len(G.nodes)),
    iterations=1000,
)

graph_renderer.node_renderer.glyph.update(
    size="size rescaled",
    fill_color="current_color",
    fill_alpha=0.8,
)

graph_renderer.edge_renderer.glyph.update(
    line_color="color", line_alpha=0.8, line_width=1
)

plot.renderers.append(graph_renderer)


# colorbars

color_bar_dict = {}

for var in coloring_variables_dict:
    if coloring_variables_dict[var]["style"]:
        color_bar_dict[var + "_color"] = create_colorbar(
            style=coloring_variables_dict[var]["style"],
            palette=coloring_variables_dict[var]["palette"],
            low=coloring_variables_dict[var]["min"],
            high=coloring_variables_dict[var]["max"],
        )
        color_bar_dict[var + "_color"].visible = False
        color_bar_dict[var + "_color"].title = var.replace("_", " ")
        color_bar_dict[var + "_color"].title_text_font_size = "14pt"

for key in color_bar_dict:
    plot.add_layout(color_bar_dict[key], "right")

# dropdown menu
code = """ 
        
        var node_data = graph_renderer.node_renderer.data_source.data;
        var edge_data = graph_renderer.edge_renderer.data_source.data;
        for (var i = 0; i < node_data['size'].length; i++) {
            
            graph_renderer.node_renderer.data_source.data['current_color'][i] = node_data[this.item][i];
        }
        
        
        for (var key in color_bar_dict){
            color_bar_dict[key].visible = false;
        }
        
        if (this.item in color_bar_dict) {
            color_bar_dict[this.item].visible = true;

        }
        
        graph_renderer.node_renderer.data_source.change.emit();
        graph_renderer.edge_renderer.data_source.change.emit();
        
        for (var key in color_bar_dict){
            color_bar_dict[key].change.emit();
        }


    """


callback = CustomJS(
    args=dict(graph_renderer=graph_renderer, color_bar_dict=color_bar_dict), code=code
)

menu = [(var.replace("_", " "), var + "_color") for var in coloring_variables_dict]

dropdown = Dropdown(
    label="Select a coloring function", button_type="default", menu=menu
)
dropdown.js_on_event("menu_item_click", callback)


layout = Column(dropdown, plot, sizing_mode="scale_both")

[0, 1]
[0, 1]
[0, 1]
[0, 1]
[0]
[0]
[0, 1, 2]
[0, 1, 2]


In [34]:
# show on browser
show(layout)